In [ ]:
import requests  
import os
import re
import time
import json
import random
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from pandas import DataFrame

In [ ]:
dfc=pd.read_csv('minglu/minglu2008abeijing.csv')

In [ ]:
def gettext(key,ip):
    #headers={'User-Agent':random.choice(agentpool)}
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.91 Safari/537.36'}
    url="http://zhaopin.baidu.com/company?query="+key
    #ip=random.choice(ippool)
    proxies={'http':'http://'+ip}
    print(proxies)
    try:
        result = requests.get(url, headers= headers,proxies=proxies,timeout=20) #在请求中设定头
        
    except:
        print('无法获取网页')
        return 2
    
    soup=BeautifulSoup(result.text,'lxml')
    
    if soup.title.text[:2]=='\n_' or not soup.find('span',class_="title line-clamp1"):
        return 3
    
    if soup.find('span',class_="title line-clamp1").text!=key:
        print('查询到的公司和关键词不符','key:',key,'result:',soup.find('span',class_="title line-clamp1").text)
        djson={}
        if soup.find('table',class_='gdxx'):
            djson['title']=[t.text for t in soup.find('table',class_='gdxx').find('tr').find_all('th')]
            djson['list']=[]
            for tlist in soup.find('table',class_='gdxx').find_all('tr')[1:]:
                djson['list'].append([t.text for t in tlist.find_all('td')])
        s1=soup.find('div',class_='top mb16')
        s2=soup.find('div',class_='left width-left')
        with open('E:/Py/com/2008a/beijing/'+key+'.txt','w',encoding='utf8') as f:
            f.write(str(s1)+str(s2))
        return 4
    
    else:
        djson={}
        if soup.find('table',class_='gdxx'):
            djson['title']=[t.text for t in soup.find('table',class_='gdxx').find('tr').find_all('th')]
            djson['list']=[]
            for tlist in soup.find('table',class_='gdxx').find_all('tr')[1:]:
                djson['list'].append([t.text for t in tlist.find_all('td')])
        s1=soup.find('div',class_='top mb16')
        s2=soup.find('div',class_='left width-left')
        with open('E:/Py/com/2008a/beijing/'+key+'.txt','w',encoding='utf8') as f:
            f.write(str(s1)+str(s2))
        return 1

In [ ]:
ippool=[

 '61.155.164.107:3128',
 '61.155.164.110:3128','61.155.164.111:3128',
 '180.212.140.131:8118',
 '27.194.82.159:8118',
 '175.174.122.234:80',
 '117.68.195.234:808',
 '42.176.45.184:80',
 '122.225.17.123:8080',
 '42.243.143.97:8118',
 '115.225.118.250:8118',
 '110.172.220.194:8080'
       ]

In [ ]:
len(dfc)

In [ ]:
df=pd.read_csv('qy.csv')
df2=pd.concat((df,dfc),axis=0)
dfc=df2.drop_duplicates(['company'])[len(df):].reset_index(drop=True)

In [ ]:
len(dfc)

In [ ]:
m=0
wtimes=0
i=104942

while i<233361:
    keyword=dfc.loc[i,'company']
    ip=ippool[m]
    status=gettext(keyword,ip)
    print(i,status)
        
    if status==2:
        i-=1
        wtimes+=1
        if wtimes>4:
            m+=1
            if m==len(ippool):
                print('ip用完')
                break
            wtimes=0
    else:
        wtimes=0
    i+=1

In [ ]:
dfc

In [ ]:
dfc.to_csv('minglu/beijing.csv',index=False,encoding='utf-8')